In [25]:
import pandas as pd
import numpy as np
import scipy.stats as sc

## Задача 1. Отличия до эксперимента
  
Возьмите те же группы, что и в эксперименте с изменением дизайна сайта, и проверьте значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).

В качестве ответа введите p-value, округлённое до 3-го знака после точки.

In [2]:
groups = pd.read_csv('experiment_users.csv')

In [79]:
sales = pd.read_csv('2022-04-01T12_df_sales.csv')

In [80]:
sales['date'] = pd.to_datetime(sales['date'])

In [81]:
new_sales = sales[(sales['date'] >= '2022-03-16') & (sales['date'] < '2022-03-23')]

In [82]:
df = pd.merge(groups, new_sales, on='user_id', how='left').fillna(0)

In [83]:
df

,user_id,pilot,sale_id,date,count_pizza,count_drink,price
0,0ffc65,0,0.0,0,0.0,0.0,0.0
1,b962b9,0,0.0,0,0.0,0.0,0.0
2,7ea63f,0,0.0,0,0.0,0.0,0.0
3,7f9a61,0,0.0,0,0.0,0.0,0.0
4,459e55,0,1170172.0,2022-03-22 18:12:12,3.0,1.0,2160.0
...,...,...,...,...,...,...,...
23389,b10ce5,1,0.0,0,0.0,0.0,0.0
23390,c5486d,1,0.0,0,0.0,0.0,0.0
23391,53bf9f,1,0.0,0,0.0,0.0,0.0
23392,405a2b,1,0.0,0,0.0,0.0,0.0


In [84]:
df = df.groupby('user_id', as_index=False).agg(revenue=('price', 'sum'), pilot=('pilot', 'min'))

In [85]:
stat, p_val = sc.ttest_ind(df[df['pilot'] == 0].revenue, df[df['pilot'] == 1].revenue)

In [86]:
round(p_val, 3)

0.199

## Задача 2. Среднее время между покупками
  

При выполнении прошлого задания вы могли заметить, что в данных много нулей. Это значит, что большая часть пользователей, совершивших покупку во время эксперимента, не совершала покупок на неделе до эксперимента. Интересно, как часто наши клиенты делают покупки?

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.

В качестве ответа введите среднее количество дней между покупками, округлённое до целого значения.

In [117]:
more_two = sales.groupby('user_id', as_index=False).agg(buy_count=('sale_id', 'count')).query('buy_count > 1')

In [118]:
df = sales.query('user_id in @more_two.user_id').sort_values(['user_id', 'date'])

In [119]:
min_time = df.groupby('user_id').agg({'date':'min'}).date
max_time = df.groupby('user_id').agg({'date':'max'}).date
transactions_count = df.groupby('user_id').agg({'date':'count'}).date

In [134]:
round(sum((max_time - min_time).dt.days)/(sum(transactions_count) - len(df.groupby('user_id').date)))

17